# 1. Обрезка (Pruning)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.utils.prune as prune

# Пример простой модели
class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(100, 50)

    def forward(self, x):
        return self.fc(x)

model = SimpleModel()

# Применим обрезку весов: удалим 30% наименьших по модулю
prune.l1_unstructured(model.fc, name="weight", amount=0.3)

# Проверим, что веса обрезаны (замаскированы)
print(model.fc.weight)

Структурная обрезка

In [ ]:
!pip install torch-pruning

In [ ]:
import torch
import torch.nn as nn
import torch_pruning as tp

model = nn.Sequential(
    nn.Linear(100, 64),
    nn.ReLU(),
    nn.Linear(64, 10)
)

example_input = torch.randn(1, 100)

# Создаём граф зависимостей
DG = tp.DependencyGraph()
DG.build_dependency(model, example_input=example_input)

# Выбираем слой и удаляем 30% выходных нейронов
layer_to_prune = model[0]
pruning_idxs = tp.strategy.RandomStrategy()(
    layer_to_prune.weight, amount=0.3
)

# Применяем обрезку
DG.prune(layer_to_prune, pruning_idxs)

Продвинутая структурная обрезка

In [2]:
!pip install -e git+https://github.com/marcoancona/TorchPruner.git#egg=torchpruner

ERROR: Operation cancelled by user


KeyboardInterrupt: 

In [ ]:
from torchpruner.attributions import RandomAttributionMetric
from torchpruner.pruner import Pruner

attr = RandomAttributionMetric(model, data_generator, criterion, device)
pruner = Pruner(model, attr)

# Применим обрезку к слоям
pruner.prune_layer(model.fc1, amount=0.3)

# 2. Квантование (Quantization)

PTQ (Post-Training Quantization) — это метод квантования нейросетей, который применяется после завершения обучения модели.

In [ ]:
import tensorflow as tf

# Загрузка модели
model = tf.keras.models.load_model('model.h5')

# Конвертация модели с использованием PTQ
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Сохранение квантованной модели
with open('model_quantized.tflite', 'wb') as f:
    f.write(tflite_model)

PTQ в PyTorch

In [ ]:
import torch
import torch.quantization

# Загрузка обученной модели
model = MyPretrainedModel()

# Подготовка модели к квантованию
model.eval()
model.qconfig = torch.quantization.get_default_qconfig('fbgemm')
torch.quantization.prepare(model, inplace=True)

# Применение PTQ
torch.quantization.convert(model, inplace=True)

# Тестирование квантованной модели
output = model(input_data)

# 3. Дистилляция знаний (Knowledge Distillation)

In [ ]:
!pip install KD-Lib

In [ ]:
import torch
import torch.optim as optim
from torchvision import datasets, transforms
from KD_Lib.KD import VanillaKD

# Загрузка данных (MNIST)
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_loader = torch.utils.data.DataLoader(datasets.MNIST("mnist_data", train=True, download=True, transform=transform), batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(datasets.MNIST("mnist_data", train=False, transform=transform), batch_size=32, shuffle=False)

# Определим модели
teacher_model = ...  # например, ResNet18
student_model = ...  # например, более лёгкая CNN

teacher_optimizer = optim.SGD(teacher_model.parameters(), lr=0.01)
student_optimizer = optim.SGD(student_model.parameters(), lr=0.01)

# Инициализация дистилляции
distiller = VanillaKD(teacher_model, student_model, train_loader, test_loader, teacher_optimizer, student_optimizer)

# Обучение
distiller.train_teacher(epochs=5)
distiller.train_student(epochs=5)
distiller.evaluate(teacher=False)  # Оценка студента


In [ ]:
!pip install torchdistill

In [ ]:
import torch
from torchvision import models
from torchdistill.core.forward_hook import ForwardHookManager

# Teacher-модель
teacher = models.resnet18(pretrained=True)
student = models.resnet18(pretrained=False)

# Hook для извлечения признаков
hook_mgr = ForwardHookManager(device='cpu')
hook_mgr.add_hook(teacher, 'layer1.0.bn2', requires_output=True)
hook_mgr.add_hook(student, 'layer1.0.bn2', requires_output=True)

# Пример входа
x = torch.rand(32, 3, 224, 224)
teacher_out = teacher(x)
student_out = student(x)

# Извлечённые признаки
features = hook_mgr.pop_io_dict()
teacher_feat = features['layer1.0.bn2']['output']
student_feat = features['layer1.0.bn2']['output']

# Можно использовать MSE между признаками как distillation loss
loss = torch.nn.functional.mse_loss(student_feat, teacher_feat)


# 4. Факторизация низкого ранга (Low-Rank Factorization)

PyTorch

In [ ]:
import torch

# Пример: матрица весов A размером (m x n)
A = torch.randn(1000, 1000)

# SVD-разложение
U, S, Vh = torch.linalg.svd(A, full_matrices=False)

# Выбор ранга r
r = 100
U_r = U[:, :r]
S_r = S[:r]
Vh_r = Vh[:r, :]

# Аппроксимация исходной матрицы
A_low_rank = U_r @ torch.diag(S_r) @ Vh_r

TensorFlow

In [ ]:
import tensorflow as tf

# Матрица весов
A = tf.random.normal((1000, 1000))

# SVD-разложение
s, u, v = tf.linalg.svd(A, full_matrices=False)

# Ранжирование
r = 100
A_low_rank = tf.matmul(u[:, :r], tf.matmul(tf.linalg.diag(s[:r]), tf.transpose(v[:, :r])))


PyMF

In [ ]:
# Установка библиотеки
!pip install pymf3

from pymf import SVD

# SVD-факторизация
data = np.random.rand(1000, 1000)
model = SVD(data, num_bases=100)
model.factorize()

A_approx = model.W @ model.H